<a href="https://colab.research.google.com/github/kchenTTP/ors_test_grading_automation/blob/main/ORS_Grading_Assessments_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORS Assessment Test Automatic Grader
Automate grading assessment tests and output a report in excel format

**Requirments**
1. Get the correct answers of the assessment
1. Filter by student (also filter by datetime for current cohort)
1. Find the pre-class assessment
1. Find the post-class assessment
1. Find reattempts of tests
1. Show only answers of the questions students get wrong
1. Transpose from wide data to long data
  1. Column: Attempts (labels: pre-class test, post-class test, reattempts)
  1. Row: Scores, Questions 1 - 20
1. Save into `.xlsx` file

**Tasks**
- Provide dataset of the students (`.csv` or google sheets api)
- Provide student names

## Install Required Libraries & Import

In [ ]:
!pip install weasyprint

In [2]:
import pandas as pd
import numpy as np
from weasyprint import HTML, CSS

## Create Folders & Upload Files
- Assessment test responses `.csv` file from this [link](https://drive.google.com/drive/folders/142C-KrYeCN2GnEUiyou6cvDYIOyOWKkt?usp=drive_link)

- Student information from this [link](https://nyplorg-my.sharepoint.com/:x:/g/personal/kangchen_nypl_org/EVcsHhpkqM9Fteg3pnvZBZoBRawkm43iJZF-3YE5hgRyCA?email=kangchen%40nypl.org&e=l5Vz5w)

In [3]:
!mkdir data output

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘output’: File exists


## Set Variables

In [130]:
test_dates =['2023-09-16', '2023-10-14']
test_dates = [pd.to_datetime(date) for date in test_dates]
program = 'word'

## Process Names

In [5]:
# set variables
df = pd.read_csv('./data/ors_info.csv')
df

,FirstName,LastName,Drop,Email,OneDrive,MosCertInterest,WordTest
0,Alana,Chuong,NaN,alanachuong@gmail.com,alanachuong.works@gmail.com,True,F
1,Alison,Wong,NaN,awtextmessages@gmail.com,awtextmessages@gmail.com,True,F
2,Alona Brequillo,Alona Brequillo,NaN,brequillo.alona@gmail.com,brequillo.alona@gmail.com,True,F
3,Beverly,Sealey,NaN,bevlun7@gmail.com,bevlun36@hotmail.com,True,-
4,Cherie,Pepper,NaN,pepper_cherie@yahoo.com,cherie.pepper@yahoo.com,True,P
5,Eleanor,Sasso,NaN,candidateinformation@me.com,infoc7012@gmail.com,True,P
6,Elsa,Wilson,NaN,elsa.wilson220@gmail.com,elsa.wilson220@gmail.com,True,-
7,FELICIA,POOLER,NaN,financialpeace312@gmail.com,poolerfelicia1@outlook.com,True,P
8,Fran,Brown,NaN,franaltabrown@gmail.com,franaltamirbrown@outlook.com,True,F
9,John,Niland,NaN,johnnyspotone@gmail.com,JohnNiland100@outlook.com,True,F


### Preprocess names

In [6]:
# drop unactive students
df = df.loc[df.Drop != True]

In [7]:
student_info = df[['FirstName', 'LastName', 'Email']]

In [8]:
names = df[['FirstName', 'LastName']]

In [9]:
names.loc[names.FirstName == 'Eleanor', ['FirstName']] = 'E'

<ipython-input-9-329b505e6274>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.loc[names.FirstName == 'Eleanor', ['FirstName']] = 'E'


In [10]:
# fix error values firstname = lastname
names.iloc[2,0] =names.iloc[2][0].split(' ')[0]
names.iloc[2,1] =names.iloc[2][1].split(' ')[1]

<ipython-input-10-e490942b9c8e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.iloc[2,0] =names.iloc[2][0].split(' ')[0]
<ipython-input-10-e490942b9c8e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.iloc[2,1] =names.iloc[2][1].split(' ')[1]


In [11]:
names.FirstName = names.FirstName.str.lower().str.strip()
names.LastName = names.LastName.str.lower().str.replace(' ', '')

<ipython-input-11-6bad5b516196>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.FirstName = names.FirstName.str.lower().str.strip()
<ipython-input-11-6bad5b516196>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names.LastName = names.LastName.str.lower().str.replace(' ', '')


In [12]:
fullnames = pd.DataFrame(names.FirstName + ' ' + names.LastName, columns=['fullname'])

In [13]:
fullnames

,fullname
0,alana chuong
1,alison wong
2,alona brequillo
3,beverly sealey
4,cherie pepper
5,e sasso
6,elsa wilson
7,felicia pooler
8,fran brown
9,john niland


## Process Responses

In [14]:
assess_df = pd.read_csv('./data/ORS_Word_Assessment_Responses.csv')
assess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 25 columns):
 #   Column                                                                                                                                                                                                                                                     Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                     --------------  -----  
 0   Timestamp                                                                                                                                                                                                                                                  899 non-null    object 
 1   Email Address                                     

In [15]:
# rename columns
col_names_to_replace = list(assess_df.columns)
col_names = ['timestamp', 'email', 'score', 'firstname', 'lastname']

for i in range(1,21):
  col_names.append(f'Q{i}')

col_mapper = dict.fromkeys(col_names_to_replace)
for i, col in enumerate(col_names_to_replace):
  col_mapper[col] = col_names[i]

assess_df.rename(columns=col_mapper, inplace=True)

### Preprocess first and last name

In [16]:
assess_df.firstname = assess_df.firstname.str.lower()
assess_df.lastname = assess_df.lastname.str.lower()
assess_df.head(5)

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,9/15/2020 18:09:55,cmorrala1@gmail.com,70 / 100,cinthya,orrala,Insert a Page Break.,She can change the page numbering for at least...,Select the text and make edits.,True,Carmen needs to select the entire document and...,...,PowerPoint,True,Creating a flyer,True,False,Save it as a pdf.,True,0.25,Click Add to Dictionary.,She can click on Header & Footer from the Inse...
1,9/14/2020 16:17:29,grosso.sharon28@gmail.com,70 / 100,sharon,grosso,Insert a Page Break.,She can change the orientation for at least on...,Select the text and make edits.,True,Carmen needs to select the entire document and...,...,Excel,True,Creating a flyer,True,False,Save it as a pdf.,True,1.00,Click Add to Dictionary.,She can click on Header & Footer from the Inse...
2,9/14/2020 16:18:09,ooaakn@gmail.com,60 / 100,natanya,oakley,Insert a Page Break.,I don't know.,Use highlighter to highlight the text then del...,True,Carmen needs to select the entire document and...,...,Excel,True,Bulk Mail,True,False,Save it as a pdf.,False,0.25,Click AutoCorrect.,She can click on Header & Footer from the Inse...
3,9/14/2020 16:18:30,karnoldnyc@gmail.com,75 / 100,katie,arnold,Insert a Page Break.,She can change the margins for at least one pa...,Select the text and make edits.,True,Style Sets are only applied to the relevant st...,...,Excel,True,Creating a flyer,True,False,Save it as a pdf.,False,0.25,Click Change All.,She can click on Header & Footer from the Inse...
4,9/14/2020 16:20:51,fettmartin@gmail.com,55 / 100,martin,fett,Insert a Page Break.,I don't know.,Select the text and make edits.,True,Carmen needs to select the entire document and...,...,Word,True,Creating a flyer,False,True,Save it as a pdf.,True,0.50,Click AutoCorrect.,"She can type ""Fiscal Year Report"" at the top o..."


In [17]:
# correct error values: elsa divinagracia = elsa
assess_df.loc[(assess_df.lastname == 'wilson') & (assess_df.firstname == 'elsa divinagracia'), 'firstname'] = 'elsa'

### Get Answers

In [18]:
answer_row = assess_df[assess_df.score == '100 / 100'].tail(1).reset_index(drop=True)
answer_row.iloc[:,:5] = np.nan
answer_key = answer_row.iloc[:,5:]
answer_dict = answer_key.to_dict(orient='records')[0]

<ipython-input-18-4c9b7091278b>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  answer_row.iloc[:,:5] = np.nan


In [19]:
answer_dict

{'Q1': 'Insert a Page Break.',
 'Q2': 'She can change the margins for at least one page., She can change the character spacing for at least one page., She can change the orientation for at least one page., She can change the page numbering for at least one page.',
 'Q3': 'Select the text and make edits.',
 'Q4': True,
 'Q5': "Style Sets are only applied to the relevant styles in the document. Carmen's document contains only the normal style so there are no title/heading/subtitle styles to apply the style set to.",
 'Q6': 'All of the above',
 'Q7': 'Click on the picture in your document.',
 'Q8': 'Format Painter',
 'Q9': 'Clicking the Increase Indent button., Pressing the Tab key., Dragging the indent marker to the right.',
 'Q10': 'Click on the Layout Ribbon under Table Tools and click Insert Below twice.',
 'Q11': 'Track Changes',
 'Q12': True,
 'Q13': 'Creating a beautiful flyer',
 'Q14': True,
 'Q15': False,
 'Q16': 'Save it as a PDF.',
 'Q17': True,
 'Q18': 0.5,
 'Q19': 'Click Auto

### Get Data Base on Test Date and Student Name

In [20]:
# convert timestamp to datetime dtype
assess_df.timestamp = pd.to_datetime(assess_df.timestamp, format='%m/%d/%Y %H:%M:%S', errors='coerce')

In [21]:
assess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  898 non-null    datetime64[ns]
 1   email      899 non-null    object        
 2   score      899 non-null    object        
 3   firstname  899 non-null    object        
 4   lastname   899 non-null    object        
 5   Q1         899 non-null    object        
 6   Q2         899 non-null    object        
 7   Q3         899 non-null    object        
 8   Q4         899 non-null    bool          
 9   Q5         899 non-null    object        
 10  Q6         899 non-null    object        
 11  Q7         899 non-null    object        
 12  Q8         898 non-null    object        
 13  Q9         899 non-null    object        
 14  Q10        877 non-null    object        
 15  Q11        899 non-null    object        
 16  Q12        899 non-null    bool          
 1

In [22]:
# get all tests from test dates
all_word_assessment = assess_df[assess_df.timestamp.dt.date.isin([d.date() for d in test_dates])]

In [23]:
# check student names and test count
all_word_assessment[all_word_assessment.firstname.isin(names.FirstName)].firstname.value_counts().sort_index()

alana        2
alison       2
alona        2
beverly      1
cherie       2
e            2
elsa         1
felicia      2
fran         2
john         2
julia        1
mayra        2
myrna        1
orlando      2
robert       2
roselynne    2
shirley      1
sitara       1
terre        1
Name: firstname, dtype: int64

In [24]:
# all section 4 word test data
sect4_word_test = all_word_assessment[all_word_assessment.firstname.isin(names.FirstName)]
sect4_word_test.reset_index(drop=True, inplace=True)

## Generate Excel Report

### Filter Incorrect Answers

In [25]:
info_df = sect4_word_test.iloc[:,:5] # student info
responses_df = sect4_word_test.iloc[:,5:] # student response
wrong_answer_filter = responses_df != pd.concat([answer_key] * 31, ignore_index=True)
wrong_answer_df = responses_df[wrong_answer_filter] # retain answer values that are incorrect

In [26]:
# final dataframe with all student information and the questions the got wrong
final_results = pd.concat([info_df, wrong_answer_df], axis=1)

In [27]:
final_results.head(2)

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,2023-09-16 11:31:25,romixyz@gmail.com,85 / 100,roselynne,mizner,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Click Add to Dictionary.,NaN
1,2023-09-16 11:34:26,poolerfelicia1@outlook.com,80 / 100,felicia,pooler,NaN,NaN,NaN,NaN,Carmen needs to select the entire document and...,...,Comments,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN


### Save All Results to Dictionary

In [28]:
cols_to_show_list = []
grades_dict = {}
cols_to_drop = ['timestamp', 'email', 'score', 'firstname', 'lastname']

# iterate through each row
for i, row in final_results.iterrows():
  cols = list(final_results.columns[row.notna()]) # columns that don't contain null values
  cols_to_show_list.append(cols)

  # student answers
  stu_name = row.firstname.strip() + " " + row.lastname.strip()
  test_time = str(row.timestamp)

  # check if name exists
  if grades_dict.get(stu_name) == None:
    grades_dict[stu_name] = {}
  if grades_dict[stu_name].get(test_time) == None:
    grades_dict[stu_name][test_time] = {
        'score': row.score.split(' / ')[0],
        'res': pd.DataFrame(row.loc[cols]).T.drop(columns=cols_to_drop),
        'ans': answer_row[cols].drop(columns=cols_to_drop)
    }
  else:
    print(row.timestamp)

In [29]:
grades_dict.keys()

dict_keys(['roselynne mizner', 'felicia pooler', 'fran brown', 'cherie pepper', 'shirley pendleton', 'e sasso', 'mayra lindemann', 'elsa wilson', 'john niland', 'alana chuong', 'sitara shamim', 'robert vargas', 'myrna gabriel', 'orlando roman', 'terre grilli', 'alison wong', 'beverly sealey', 'alona brequillo', 'julia hernandez-mejia'])

In [30]:
list(list(grades_dict.values())[0].values())[0]['score']

'85'

In [31]:
list(list(grades_dict.values())[0].values())[0]['res']

,Q10,Q14,Q19
0,Click on the Insert Ribbon and click Add Rows ...,False,Click Add to Dictionary.


In [32]:
list(list(grades_dict.values())[0].values())[0]['ans']

,Q10,Q14,Q19
0,Click on the Layout Ribbon under Table Tools a...,True,Click AutoCorrect.


### Save Results -> DataFrame -> Excel

In [106]:
# check length of data
names.shape[0] == final_results.firstname.value_counts().count()

True

In [107]:
names.shape[0] == final_results.lastname.value_counts().count()

True

In [128]:
questions = pd.DataFrame(col_names_to_replace.copy(), index=col_names).T
questions.iloc[0,0] = 'Questions'
questions.iloc[:,1:5] = np.nan
questions

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,Questions,NaN,NaN,NaN,NaN,1. Jane is writing a book. She just finished c...,2. Why would Jane want to use a Section Break ...,"3. To edit the text in a Word document, what c...",4. True or False: You can use Styles to format...,"5. After applying the ""Elegant"" Document Forma...",...,"11. José is proofreading a document for Jenna,...",12. True or False: Mail Merge is a process in ...,"13. Typically, you can use a Mail Merge to cre...",14. True or False: Your recipient list can be ...,15. True or False: Headers are only used for d...,16. Amir wants to attach his resume to an emai...,17. True or False: Compatibility Mode is used ...,"18. By default, Word places a tab stop at ever...",19. Ralph performs a spellcheck on his documen...,"20. Brandy wants to type ""Fiscal Year Report"" ..."


In [129]:
answer_row.iloc[0,0] = 'Answers'
answer_row

,timestamp,email,score,firstname,lastname,Q1,Q2,Q3,Q4,Q5,...,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,Answers,NaN,NaN,NaN,NaN,Insert a Page Break.,She can change the margins for at least one pa...,Select the text and make edits.,True,Style Sets are only applied to the relevant st...,...,Track Changes,True,Creating a beautiful flyer,True,False,Save it as a PDF.,True,0.5,Click AutoCorrect.,She can click on Header & Footer from the Inse...


In [131]:
# concat all dataframes and save as excel file
for i, row in names.iterrows():
  fname = row.FirstName
  lname = row.LastName

  cols = final_results.columns[final_results[final_results.firstname == fname].notna().any()]
  report = pd.concat([questions, answer_row, final_results[final_results.firstname == fname]], axis=0).reset_index(drop=True)[cols]
  report.fillna('-', inplace=True)
  report.drop(columns=['email', 'firstname', 'lastname'], inplace=True)
  report.rename(columns={'timestamp': 'Index'}, inplace=True)
  report.set_index('Index', inplace=True)

  report.to_excel(f'./output/{fname}_{lname}_{program}_report.xlsx', f'{fname}_{lname}')

In [132]:
# save files
!zip -r /content/grades.zip /content/output/

from google.colab import files
files.download("/content/grades.zip")

updating: content/output/ (stored 0%)
updating: content/output/elsa_wilson_word_report.xlsx (deflated 9%)
updating: content/output/alison_wong_word_report.xlsx (deflated 9%)
updating: content/output/myrna_gabriel_word_report.xlsx (deflated 9%)
updating: content/output/e_sasso_word_report.xlsx (deflated 9%)
updating: content/output/alana_chuong_word_report.xlsx (deflated 9%)
updating: content/output/terre_grilli_word_report.xlsx (deflated 9%)
updating: content/output/john_niland_word_report.xlsx (deflated 9%)
updating: content/output/mayra_lindemann_word_report.xlsx (deflated 8%)
updating: content/output/sitara_shamim_word_report.xlsx (deflated 10%)
updating: content/output/fran_brown_word_report.xlsx (deflated 9%)
updating: content/output/alona_brequillo_word_report.xlsx (deflated 8%)
updating: content/output/beverly_sealey_word_report.xlsx (deflated 9%)
updating: content/output/robert_vargas_word_report.xlsx (deflated 9%)
updating: content/output/felicia_pooler_word_report.xlsx (defla

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>